<a href="https://colab.research.google.com/github/sumankmaiti/All-in-one/blob/main/watermarking_of_ann_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import exp
from random import seed
from random import random
import tensorflow as tf
from tensorflow import keras
import numpy as np

model build

In [ ]:
# Initialize a network. Parameters are initialized using gaussian distribution
def initialize_network(inputs, outputs):
  network = list()

  hidden_layer_1 = [{'weights':[np.random.normal(0.0, .01) for i in range(inputs + 1)]} for i in range(7)]
  network.append(hidden_layer_1)

  hidden_layer_2 = [{'weights':[np.random.normal(0.0, .01) for i in range(7 + 1)]} for i in range(5)]
  network.append(hidden_layer_2)

  hidden_layer_3 = [{'weights':[np.random.normal(0.0, .01) for i in range(5 + 1)]} for i in range(3)]
  network.append(hidden_layer_3)

  output_layer = [{'weights':[np.random.normal(0.0, .01) for i in range(3 + 1)]} for i in range(outputs)]
  network.append(output_layer)
  return network

In [ ]:
# Calculate neuron activation for an input
def weighted_sum(weights, inputs):
	bias = weights[-1] # last value of the 'weights' is used for bias
	for i in range(len(weights)-1): # loop till 2nd last element
		bias += weights[i] * inputs[i] # weighted sum
	return bias

In [ ]:
# Transfer neuron activation
def sigmoid(activation):
	return 1.0 / (1.0 + exp(-activation))

In [ ]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []   # store the output of the neurons for a layer
		for neuron in layer: 
			activation = weighted_sum(neuron['weights'], inputs) # weighted sum
			neuron['output'] = sigmoid(activation)  # sigmoid activation 
			new_inputs.append(neuron['output']) # output of the neurons in a layer
		inputs = new_inputs  # make the output of a layer as input for the next layer
	return inputs # final output from output layer

In [ ]:
# Calculate the derivative of a neuron output
def derivative(output):
	return output * (1.0 - output)

In [ ]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i] # last to first
		errors = list() 
		if i != len(network)-1: # from 2nd last to first layer
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]: # 
					error += (neuron['weights'][j] * neuron['delta']) # total error for a neuron w.r.t connected neurons from back
				errors.append(error)
		else:   # 1st execute here to calculate loss (expected - predected) 
			for j in range(len(layer)): # last layer
				neuron = layer[j]
				errors.append(expected[j] - neuron['output']) # loss of output neurons
		for j in range(len(layer)):  # calculate gradient/ derivative(small portion from error w.r.t output of that neuron)
			neuron = layer[j]
			neuron['delta'] = errors[j] * derivative(neuron['output']) # store the derivative

In [ ]:
# Update network weights with error
def update_weights(network, row, l_rate, W):
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['output'] for neuron in network[i - 1]]  # from 2nd layer output of 1st= input of 2nd
    for neuron in range(len(network[i])):  # neuron in a layer
      # print(network[i][neuron])   
      for j in range(len(inputs)):
        try:
          layer_number = W[i][0]
        except:
          layer_number = -10
        if neuron == layer_number:
          try:
            weight_number = W[i][1]
          except:
            weight_number = -10
          if j == weight_number:
            # network[i][neuron]['weights'][j] += 0 * l_rate * network[i][neuron]['delta'] * inputs[j]
            network[i][neuron]['weights'][j] += 0 * l_rate * network[i][neuron]['delta']
            # print(network[i][neuron]['weights'][j])  
          else:
            # network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] * inputs[j] 
            network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] 
        else:
          # print(network[i][neuron])
          # network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] * inputs[j]
          network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] 
      # for j in range(len(inputs)):
      #   print(network[i][neuron]['weights'][j])
      #   neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
      network[i][neuron]['weights'][-1] += l_rate * network[i][neuron]['delta']

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, W):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row) # final output from output layer
      expected = [0 for i in range(n_outputs)] 
      expected[row[-1]] = 1 
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # squired sum of total error in output:  sum((y'-y) squire)
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate, W)
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
  return expected, row

In [ ]:
def test_network(network, test_data):
  # for epoch in range(n_epoch):
  #   sum_error = 0
  for row in test_data:
    outputs = forward_propagate(network, row) # final output from output layer
    # expected = [0 for i in range(n_outputs)] 
    # expected[row[-1]] = 1 
    # sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # squired sum of total error in output:  sum((y'-y) squire)
    # backward_propagate_error(network, expected)
    # update_weights(network, row, l_rate, W)
    print("outputs:",outputs)

In [ ]:
# Test training backprop algorithm
seed(10)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, n_outputs)

embed watermark

In [ ]:
# watermark
X=[0.9, 0.76, 0.48]

# selected weights from each layer to be watermarked
W= dict()
W[0]=[1,0] 
W[1]=[1,1]
W[2]=[1,2]

# update the weight located in W
for item in range(len(W)):
  layer = item
  mu = 0.1
  sigma = 0.001
  neuron_with_weight=W[item]
  value_of_selected_weight = network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]]
  print("before update layer", layer, "neuron", neuron_with_weight[0],"weight index", neuron_with_weight[1], "->", value_of_selected_weight)
  value_of_selected_weight = 2*sigma*(X[item] - 0.5) + mu
  network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]] = value_of_selected_weight
  print("after update layer ", layer, "neuron", neuron_with_weight[0],"weight index", neuron_with_weight[1], "->", network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]])

before update layer 0 neuron 1 weight index 0 -> 0.006548326221975255
after update layer  0 neuron 1 weight index 0 -> 0.1008
before update layer 1 neuron 1 weight index 1 -> 0.006313146269974237
after update layer  1 neuron 1 weight index 1 -> 0.10052000000000001
before update layer 2 neuron 1 weight index 2 -> 0.02531908498698365
after update layer  2 neuron 1 weight index 2 -> 0.09996000000000001


In [ ]:
# after updating the weight
for layer in network:
	print(layer)

[{'weights': [0.011518163119584555, 0.00036012637945079826, -0.0006068413742203497]}, {'weights': [0.1008, 0.008238627772396654, 0.0008448564850340855]}, {'weights': [0.006441188085708311, -0.009170587757917082, -0.005592826757153489]}, {'weights': [0.010108626284691178, -0.0037411235406674616, -0.005874669122090942]}, {'weights': [0.0047172652277122275, 0.0004355496136894334, -0.0017796355356050376]}, {'weights': [0.013175080920362982, -0.007364942988887214, -0.017398847540954637]}, {'weights': [-0.010833929954158297, 0.012886891197422845, 0.00046499251978211504]}]
[{'weights': [-0.016544040163402364, -0.0023141989075542665, 0.01414984305341131, -0.008955984319255796, -0.008968272212101586, -6.629927217613041e-05, 0.0017490786481609564, -0.008551237394299369]}, {'weights': [0.01402191064982743, 0.10052000000000001, 0.015002614946360916, -0.014695820432736295, 0.0004230730245053447, -0.0015703272686653398, 0.003717656325967631, -0.016813239589524874]}, {'weights': [0.01268335791645142,

train the model

In [ ]:
# train the network
expected_output, row = train_network(network, dataset, 0.1, 5000, n_outputs, W)

Streaming output truncated to the last 5000 lines.
>epoch=0, lrate=0.100, error=5.069
>epoch=1, lrate=0.100, error=5.069
>epoch=2, lrate=0.100, error=5.070
>epoch=3, lrate=0.100, error=5.070
>epoch=4, lrate=0.100, error=5.071
>epoch=5, lrate=0.100, error=5.071
>epoch=6, lrate=0.100, error=5.072
>epoch=7, lrate=0.100, error=5.072
>epoch=8, lrate=0.100, error=5.073
>epoch=9, lrate=0.100, error=5.073
>epoch=10, lrate=0.100, error=5.074
>epoch=11, lrate=0.100, error=5.074
>epoch=12, lrate=0.100, error=5.074
>epoch=13, lrate=0.100, error=5.075
>epoch=14, lrate=0.100, error=5.075
>epoch=15, lrate=0.100, error=5.075
>epoch=16, lrate=0.100, error=5.075
>epoch=17, lrate=0.100, error=5.075
>epoch=18, lrate=0.100, error=5.076
>epoch=19, lrate=0.100, error=5.076
>epoch=20, lrate=0.100, error=5.076
>epoch=21, lrate=0.100, error=5.076
>epoch=22, lrate=0.100, error=5.076
>epoch=23, lrate=0.100, error=5.076
>epoch=24, lrate=0.100, error=5.076
>epoch=25, lrate=0.100, error=5.076
>epoch=26, lrate=0.100,

In [ ]:
# after training the watermarked weights are as it is
for layer in network:
	print("after: ", layer)

after:  [{'weights': [0.03812149344356955, 0.02696345670343587, 0.02599648894976469], 'output': 0.6018462491296204, 'delta': -2.4002466453147895e-05}, {'weights': [0.1008, 0.0341064417983815, 0.02671267051101882], 'output': 0.7150216598119407, 'delta': -1.7156738342375747e-05}, {'weights': [0.03497040449457786, 0.019358628650952055, 0.022936389651715922], 'output': 0.5888569805696708, 'delta': -2.5018219206002447e-05}, {'weights': [0.035517232195041964, 0.021667482369683418, 0.01953393678825977], 'output': 0.5910082601194829, 'delta': -2.362414738136102e-05}, {'weights': [0.02929746978686172, 0.025015754172839008, 0.022800569023544506], 'output': 0.5830775792839367, 'delta': -2.3408129822324413e-05}, {'weights': [0.03910639107769908, 0.01856636716844865, 0.008532462616381221], 'output': 0.5923758280375931, 'delta': -2.382327144175214e-05}, {'weights': [0.01583508283832158, 0.0395559039899027, 0.02713400531226178], 'output': 0.5713749100636992, 'delta': -2.4627824241017852e-05}]
after: 

In [ ]:
test_network(network, [[2.7810836,2.550537003], [7.627531214,2.759262235]])

outputs: [0.49146169719256455, 0.5085374978181163]
outputs: [0.49151067820590133, 0.5084910537553583]


get watermark

In [ ]:
# selected weights from each layer to be watermarked
W= dict()
W[0]=[1,0] 
W[1]=[1,1]
W[2]=[1,2]

# update the weight located in W
for item in range(len(W)):
  layer = item
  mu = 0.1
  sigma = 0.001
  neuron_with_weight=W[item]
  value_of_selected_weight = network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]]
  print("weight of", layer, "neuron", neuron_with_weight[0],"weight index", neuron_with_weight[1], "->", value_of_selected_weight)
  watermark = ((value_of_selected_weight - mu)/ (2 * sigma)) + 0.5
  network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]] = value_of_selected_weight
  print("watermark ",item,":", round(watermark, 2))

weight of 0 neuron 1 weight index 0 -> 0.1008
watermark  0 : 0.9
weight of 1 neuron 1 weight index 1 -> 0.10052000000000001
watermark  1 : 0.76
weight of 2 neuron 1 weight index 2 -> 0.09996000000000001
watermark  2 : 0.48


make perturbed model

In [ ]:
# save the model to disk
import pickle
filename = 'finalized_model.sav'
pickle.dump(network, open(filename, 'wb'))

In [ ]:
# perturbed models of the original 
loaded_model = pickle.load(open(filename, 'rb'))
for layer in loaded_model:
	print("perturbed model: ", layer)

perturbed model:  [{'weights': [0.03812149344356955, 0.02696345670343587, 0.02599648894976469], 'output': 0.5965639885884741, 'delta': -2.4002466453147895e-05}, {'weights': [0.1008, 0.0341064417983815, 0.02671267051101882], 'output': 0.708823395328842, 'delta': -1.7156738342375747e-05}, {'weights': [0.03497040449457786, 0.019358628650952055, 0.022936389651715922], 'output': 0.584940870307593, 'delta': -2.5018219206002447e-05}, {'weights': [0.035517232195041964, 0.021667482369683418, 0.01953393678825977], 'output': 0.586673109675671, 'delta': -2.362414738136102e-05}, {'weights': [0.02929746978686172, 0.025015754172839008, 0.022800569023544506], 'output': 0.5781766840879552, 'delta': -2.3408129822324413e-05}, {'weights': [0.03910639107769908, 0.01856636716844865, 0.008532462616381221], 'output': 0.5885676237728372, 'delta': -2.382327144175214e-05}, {'weights': [0.01583508283832158, 0.0395559039899027, 0.02713400531226178], 'output': 0.5639138573671285, 'delta': -2.4627824241017852e-05}]


add noise in the watermark

In [ ]:
# values are initialized according to main model with some noise added into watermark values
for item in range(len(W)):
  layer = item
  neuron_with_weight=W[item]
  value_of_selected_weight = loaded_model[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]]
  print("before update layer", layer, "neuron", neuron_with_weight[0],"weight index", neuron_with_weight[1], "->", value_of_selected_weight)
  value_of_selected_weight = value_of_selected_weight + np.random.normal(0,0.01)
  loaded_model[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]] = value_of_selected_weight
  print("after update layer ", layer, "neuron", neuron_with_weight[0],"weight index", neuron_with_weight[1], "->", loaded_model[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]])
# result = loaded_model.score(X_test, Y_test)

before update layer 0 neuron 1 weight index 0 -> 0.1008
after update layer  0 neuron 1 weight index 0 -> 0.08301736186626135
before update layer 1 neuron 1 weight index 1 -> 0.10052000000000001
after update layer  1 neuron 1 weight index 1 -> 0.09518078953876245
before update layer 2 neuron 1 weight index 2 -> 0.09996000000000001
after update layer  2 neuron 1 weight index 2 -> 0.10611728180864399


In [ ]:
for layer in loaded_model:
	print(layer)

[{'weights': [0.03812149344356955, 0.02696345670343587, 0.02599648894976469], 'output': 0.5965639885884741, 'delta': -2.4002466453147895e-05}, {'weights': [0.08301736186626135, 0.0341064417983815, 0.02671267051101882], 'output': 0.708823395328842, 'delta': -1.7156738342375747e-05}, {'weights': [0.03497040449457786, 0.019358628650952055, 0.022936389651715922], 'output': 0.584940870307593, 'delta': -2.5018219206002447e-05}, {'weights': [0.035517232195041964, 0.021667482369683418, 0.01953393678825977], 'output': 0.586673109675671, 'delta': -2.362414738136102e-05}, {'weights': [0.02929746978686172, 0.025015754172839008, 0.022800569023544506], 'output': 0.5781766840879552, 'delta': -2.3408129822324413e-05}, {'weights': [0.03910639107769908, 0.01856636716844865, 0.008532462616381221], 'output': 0.5885676237728372, 'delta': -2.382327144175214e-05}, {'weights': [0.01583508283832158, 0.0395559039899027, 0.02713400531226178], 'output': 0.5639138573671285, 'delta': -2.4627824241017852e-05}]
[{'we

get gradient from the perturbed model

In [ ]:
# after adding noise extract gradients of all weights from perturbed models.
def backward_propagate_error_for_perturbed_models(model, expected):
  for i in reversed(range(len(model))):
    layer = model[i] # last to first
    errors = list() 
    if i != len(model)-1: # from 2nd last to first layer
      for j in range(len(layer)):
        error = 0.0
        for neuron in model[i + 1]: # 
          error += (neuron['weights'][j] * neuron['delta']) # total error for a neuron w.r.t connected neurons from back
        errors.append(error)
    else:   # 1st execute here to calculate loss (expected - predected) 
      for j in range(len(layer)): # last layer
        neuron = layer[j]
        errors.append(expected[j] - neuron['output']) # loss of output neurons
    for j in range(len(layer)):  # calculate gradient/ derivative(small portion from error w.r.t output of that neuron)
      neuron = layer[j]
      neuron['delta'] = errors[j] * derivative(neuron['output']) # store the derivative
  return model

perturbed_network = backward_propagate_error_for_perturbed_models(loaded_model, expected_output)

In [ ]:
print("gradiens and weights of the perturbed model")
for layers in perturbed_network:
  print(layers)

gradiens and weights of the perturbed model
[{'weights': [0.03812149344356955, 0.02696345670343587, 0.02599648894976469], 'output': 0.5965639885884741, 'delta': -3.2891296325419064e-05}, {'weights': [0.08301736186626135, 0.0341064417983815, 0.02671267051101882], 'output': 0.708823395328842, 'delta': -2.3567696980334914e-05}, {'weights': [0.03497040449457786, 0.019358628650952055, 0.022936389651715922], 'output': 0.584940870307593, 'delta': -3.424854565216472e-05}, {'weights': [0.035517232195041964, 0.021667482369683418, 0.01953393678825977], 'output': 0.586673109675671, 'delta': -3.23380399275585e-05}, {'weights': [0.02929746978686172, 0.025015754172839008, 0.022800569023544506], 'output': 0.5781766840879552, 'delta': -3.2045492036999885e-05}, {'weights': [0.03910639107769908, 0.01856636716844865, 0.008532462616381221], 'output': 0.5885676237728372, 'delta': -3.259904117026103e-05}, {'weights': [0.01583508283832158, 0.0395559039899027, 0.02713400531226178], 'output': 0.5639138573671285

update the non watermarked weight

In [ ]:
# Update network weights with error
def update_weights_2(perturbed_network, network, row, l_rate, W):
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['output'] for neuron in network[i - 1]]  # from 2nd layer output of 1st= input of 2nd
    for neuron in range(len(network[i])):  # neuron in a layer
      # print(network[i][neuron])   
      for j in range(len(inputs)):
        try:
          layer_number = W[i][0]
        except:
          layer_number = -10
        if neuron == layer_number:
          try:
            weight_number = W[i][1]
          except:
            weight_number = -10
          if j == weight_number:
            # network[i][neuron]['weights'][j] += 0 * l_rate * network[i][neuron]['delta'] * inputs[j]
            network[i][neuron]['weights'][j] += 0 * l_rate * network[i][neuron]['delta'] - 0.0001 * perturbed_network[i][neuron]['delta']
            # print(network[i][neuron]['weights'][j])  
          else:
            # network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] * inputs[j] 
            network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] - 0.0001 * perturbed_network[i][neuron]['delta']
        else:
          # print(network[i][neuron])
          # network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] * inputs[j]
          network[i][neuron]['weights'][j] += l_rate * network[i][neuron]['delta'] - 0.0001 * perturbed_network[i][neuron]['delta']
      # for j in range(len(inputs)):
      #   print(network[i][neuron]['weights'][j])
      #   neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
      network[i][neuron]['weights'][-1] += l_rate * network[i][neuron]['delta'] - - 0.0001 * perturbed_network[i][neuron]['delta']

In [ ]:
update_weights_2(perturbed_network, network, row, 0.1, W)

In [ ]:
for layers in network:
  print(layers)

[{'weights': [0.03811909648605387, 0.026961059745920186, 0.025994085413989742], 'output': 0.5965639885884741, 'delta': -2.4002466453147895e-05}, {'weights': [0.1008000023567697, 0.03410472848131696, 0.02671095248041488], 'output': 0.708823395328842, 'delta': -1.7156738342375747e-05}, {'weights': [0.034967906097511825, 0.01935613025388602, 0.022933884404940756], 'output': 0.584940870307593, 'delta': -2.5018219206002447e-05}, {'weights': [0.03551487301410782, 0.021665123188749275, 0.01953157113971764], 'output': 0.586673109675671, 'delta': -2.362414738136102e-05}, {'weights': [0.029295132178428693, 0.02501341656440598, 0.02279822500601307], 'output': 0.5781766840879552, 'delta': -2.3408129822324413e-05}, {'weights': [0.03910401201045902, 0.018563988101208594, 0.008530077029332929], 'output': 0.5885676237728372, 'delta': -2.382327144175214e-05}, {'weights': [0.01583262342926849, 0.039553444580849605, 0.02713153915646667], 'output': 0.5639138573671285, 'delta': -2.4627824241017852e-05}]
[{

In [ ]:
for layers in perturbed_network:
  print(layers)

[{'weights': [0.03812149344356955, 0.02696345670343587, 0.02599648894976469], 'output': 0.5965639885884741, 'delta': -3.2891296325419064e-05}, {'weights': [0.08301736186626135, 0.0341064417983815, 0.02671267051101882], 'output': 0.708823395328842, 'delta': -2.3567696980334914e-05}, {'weights': [0.03497040449457786, 0.019358628650952055, 0.022936389651715922], 'output': 0.584940870307593, 'delta': -3.424854565216472e-05}, {'weights': [0.035517232195041964, 0.021667482369683418, 0.01953393678825977], 'output': 0.586673109675671, 'delta': -3.23380399275585e-05}, {'weights': [0.02929746978686172, 0.025015754172839008, 0.022800569023544506], 'output': 0.5781766840879552, 'delta': -3.2045492036999885e-05}, {'weights': [0.03910639107769908, 0.01856636716844865, 0.008532462616381221], 'output': 0.5885676237728372, 'delta': -3.259904117026103e-05}, {'weights': [0.01583508283832158, 0.0395559039899027, 0.02713400531226178], 'output': 0.5639138573671285, 'delta': -3.373371011954763e-05}]
[{'weigh

get the watermark

In [ ]:
# get watermark

# selected weights from each layer to be watermarked
W= dict()
W[0]=[1,0] 
W[1]=[1,1]
W[2]=[1,2]

# update the weight located in W
for item in range(len(W)):
  layer = item
  mu = 0.1
  sigma = 0.001
  neuron_with_weight=W[item]
  value_of_selected_weight = network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]]
  print("weight of", layer, "neuron", neuron_with_weight[0],"weight index", neuron_with_weight[1], "->", value_of_selected_weight)
  watermark = ((value_of_selected_weight - mu)/ (2 * sigma)) + 0.5
  network[layer][neuron_with_weight[0]]['weights'][neuron_with_weight[1]] = value_of_selected_weight
  print("watermark ",item,":", round(watermark, 2))

weight of 0 neuron 1 weight index 0 -> 0.1008000023567697
watermark  0 : 0.9
weight of 1 neuron 1 weight index 1 -> 0.10052001430980317
watermark  1 : 0.76
weight of 2 neuron 1 weight index 2 -> 0.09995996913158048
watermark  2 : 0.48


In [ ]:
test_network(network, [[2.7810836,2.550537003], [7.627531214,2.759262235]])

outputs: [0.488092040940623, 0.5119071327655171]
outputs: [0.48815370656275653, 0.511848003541458]
